# 1. Draw an ASCII ART Christmas tree in Python style (do not import any library)

In [1]:
def test_tree(start,end,width=None):
    if width == None: width = end
    tree_len = 2*width+1
    for i in range(start,end):
        leaf = '*'*(2*i+1)
        print(leaf.center(tree_len))

total_len = 10
section = [(0,7),(4,7),(5,9)]
for start,end in section: test_tree(start,end,total_len)
# 樹幹
center_len = 2*total_len+1
for i in range(3): print('***'.center(center_len))

          *          
         ***         
        *****        
       *******       
      *********      
     ***********     
    *************    
      *********      
     ***********     
    *************    
     ***********     
    *************    
   ***************   
  *****************  
         ***         
         ***         
         ***         


# 2. Please use Python multiple processes(or threads) to create a class to start 3 journeys and print out 1, 2 and 3 processes respectively.

In [2]:
import threading

class test():
    def __init__(self):
        # 物件建立後隨即建立3個thread
        self.job_list = [threading.Thread(target=self.print_job,args=(i,)) for i in range(1,3+1)]
    
    def print_job(self,n):
        print('thread {}'.format(n))
    
    def start_jobs(self):
        # 啟動thread
        for tj in self.job_list: tj.start()
        # 等待所有thread執行完畢
        for tj in self.job_list: tj.join()
        print('done')

t = test()
t.start_jobs()

thread 1
thread 2
thread 3
done


# 3. What is pypy? (in 10 words)

A : python應用程式的直譯器

# 4. Try to finish this project via Python:
Use Wikipedia Simplified / Traditional Chinese conversion tables
https://phabricator.wikimedia.org/source/mediawiki/browse/master/languages/data/ZhConversion.php
Write a crawler to crawl below EC site, translate product title to Simplified Chinese, print title and price out in one raw.
https://shopee.tw/%E7%8E%A9%E5%85%B7-cat.75.2185?brands=5005&locations=-1&page=0&ratingFilter=4


## part 1 : 整理字體轉換dict

In [3]:
import requests
import json

# 將繁簡對照整理成json檔

url = 'https://phab.wmfusercontent.org/file/data/onmszjwl4lurdgdsmsxi/PHID-FILE-3vsdvdc6qmiia5yg5ask/ZhConversion.php'
res = requests.get(url)
resp_str = res.text.split('\n')

with open('output.json','w',encoding='utf8') as f:
    output_dict = {}
    for string_line in resp_str:
        if '=>' in string_line:
            sl = string_line.replace("'",'')
            sl_list = sl[:-1].split(' => ')
            try:
                # 如果後者為繁體，則會將繁體字作為key，否則會進入except
                signal = sl_list[1].decode('big5')
                output_dict[sl_list[1]] = sl_list[0]
            except:
                output_dict[sl_list[0]] = sl_list[1]
    
    # 將Dict輸出成json檔案
    json.dump(output_dict,f,indent=4)

## part 2 : 爬取商品資訊且轉換標題

### 分析網站
本次爬蟲目標為蝦皮購物的玩具商品
網址為https://shopee.tw/%E7%8E%A9%E5%85%B7-cat.75.2185?brands=5005&locations=-1&page=0&ratingFilter=4 \
經觀察後，商品的資料來源為https://shopee.tw/api/v2/search_items/
透過傳遞**brands**, **locations**, **page**及**ratingFilter**等主參數來獲得符合條件的商品。\
而requests的對象則為https://shopee.tw/api/v2/search_items/?brandids=5005&by=relevancy&fe_categoryids=2185&limit=50&locations=-1&newest=0&order=desc&page_type=search&rating_filter=4&version=2

In [6]:
import requests
import json

# 載入先前處理好的繁簡對照json
with open('output.json','r',encoding='utf-8') as f:
    translate_json = json.load(f)

url = 'https://shopee.tw/api/v2/search_items/?brandids=5005&by=relevancy&fe_categoryids=2185&limit=50&locations=-1&newest=0&order=desc&page_type=search&rating_filter=4&version=2'

# 設置header
headers = {
    'referer': 'https://shopee.tw/%E7%8E%A9%E5%85%B7-cat.75.2185?brands=5005&locations=-1&page=0&ratingFilter=4',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
}
res = requests.get(url,headers=headers)
data_json = json.loads(res.text)
for d in data_json['items']:
    title = d['name']
    price = d['price']/100000
    price_min = d['price_min']/100000
    price_max = d['price_max']/100000
    for s in title:
        # 如果標題文字存在於轉換字典中，則將繁體字轉換成簡體字
        if s in translate_json: title = title.replace(s,translate_json[s])
    print('價格: {}'.format(price),'最低價: {}'.format(price_min),'最高價: {}'.format(price_max),'標題: {}'.format(title))

價格: 198.0 最低價: 198.0 最高價: 980.0 標題: 现货 TOMY 口袋妖怪 宠物小精灵神奇宝贝精灵球皮卡丘小火龙杰尼龟 8cm触碰可翻转 爆炸精灵球整玩具 时尚最夯小物
價格: 237.0 最低價: 237.0 最高價: 837.0 標題: ‘现货’ 神奇宝贝 扭蛋 伊布皮卡丘 伊布 皮卡丘 呆河马 可达鸭 呆呆兽 变装
價格: 99.0 最低價: 99.0 最高價: 299.0 標題: 空运 TOMY口袋妖怪小火龙杰尼龟妙蛙种子雷丘皮丘皮卡丘手办
價格: 249.0 最低價: 249.0 最高價: 249.0 標題: 佳佳玩具 ----- 日本进口 神奇宝贝 宝可梦 Pokemon 手表 数码手表 卡通手表 【05391197】
價格: 50.0 最低價: 50.0 最高價: 100.0 標題: W1111 宝可梦 食玩 7-11 限定 磁铁 皮卡丘杰尼龟妙蛙种子泪眼蜥炎兔儿胖丁暴鲤龙敲音猴 不含软糖 现货
價格: 350.0 最低價: 350.0 最高價: 1680.0 標題: 现货TOMY 神奇宝贝宝可梦Pokemon Go胡帕z神固拉多白龙黑龙X黑酋雷姆裂空座盖欧卡MEGA海皇牙固拉多模型手办
價格: 20.0 最低價: 20.0 最高價: 40.0 標題: 宝可梦 全新公仔 只拆开 耿鬼 谜拟Q 呆呆兽 可达鸭 伊布 鲤鱼王 波加曼 皮卡丘 妙蛙种子 太阳伊布 杰尼龟 小火龙
價格: 250.0 最低價: 250.0 最高價: 250.0 標題: §小俏妞部屋§ [现货/单款区] 日版POLYGO 千值练 精灵宝可梦 皮卡丘耿鬼小火龙杰尼龟妙蛙种子呆呆兽 盒玩
價格: 220.0 最低價: 220.0 最高價: 999.0 標題: 【DS变装皮卡丘 黄昏之鬃、拂晓之翼、奈克洛兹玛、胡帕、波尔凯尼恩】立体图鉴GK◆精灵宝可梦/神奇宝贝★湖畔蓝鳄☆
價格: 200.0 最低價: 200.0 最高價: 220.0 標題: 一番赏 伊布 宝可梦 EIEVUI&CRYSTAL DROPS G赏F赏
價格: 105.0 最低價: 105.0 最高價: 105.0 標題: 百变怪 第七弹 P7 扭蛋 转蛋 宝可梦 神奇宝贝中心 限定
價格: 120.0 最低價: 120.0 最高價: 120.0 標題: 跳楼拼ㄌ 赔钱卖日本进口全新正版  精灵 宝